# Project 1 Part 4 - Creating a master parcel database

In this part of the project, we will use Python to read, process, and double all of the parcel data into a database.  Note that this is not our only alternative, and in Project 1 Part 4 b, we will look at another alternative, that is reading all the of original, raw files into their own database table, then using SQL to join/link/aggregate the tables.

## Chunking Files in Pandas – Part 1 (20 Points)

In this part of the project, you will use `Panda`’s to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  The data can be found at the [MinneMUDAC site](http://minneanalytics.org/minnemudac/data/).  You should document your work in a Jupyter notebook, which will be used to submit your solution.  **For the rest of the parts of this project, we will limit ourselves to the years 2004-2014.**

1. Remind me why we want to skip 2003.<br>
=> Because the columns are not consistent 


2. Import the common columns list and translation dictionaries from the `.py` file you created in the last part of the project.

In [4]:
from project_data_khanal import common_columns,lat_long_distance_dict

In [27]:
len(common_columns)

70

3. Use glob and a list comprehension to get a list of file names for the years 2004-2014.

In [28]:
from project_data_khanal import filePaths
filePaths = ['./Data/MinneMUDAC/2015_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2009_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2007_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2011_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2005_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2013_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2014_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2008_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2010_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2006_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2012_metro_tax_parcels.txt',
 './Data/MinneMUDAC/2004_metro_tax_parcels.txt']




4. Use the first chunk of the first file to prototype an expression that <br>
    a. Selects the common columns <br>
    b. Fixes any issues with the column names <br>
    c. Changes columns to the correct types (if necessary).  More information about the columns can be found [here](ftp://ftp.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_prcls_open/metadata/metadata.html). It is **imperative** that you keep the lat and long columns as strings. <br>
    d. Use the translation dictionaries from the last part to add three new columns to the chunk: lake code, lake name, parcel distance to the lake.
    e. Filters to only properties that are within 1600 m (~1 mile) of the closest lake.

In [120]:
from project_data_khanal import read_chunk
import pandas as pd
df_chunks = pd.read_csv(filePaths[0],chunksize = 1000,sep = "|")


from functoolz import first
first_chunk = first(df_chunks)
first_chunk.columns

from project_data_khanal import lake_code_dict, lat_long_id__lake_dict,lat_long_distance_dict
from dfply import *
from more_dfply import *
from itertools import zip_longest

fix_names(first_chunk)

first_chunk = (first_chunk >> select(list(common_columns)) 
             >>fix_names(make_lower=True)
             >>mutate(latitude = X.centroid_lat.astype("float").round(5).astype('object'))
             >>mutate(longitude = X.centroid_long.astype("float").round(5).astype('object'))
            >>drop(X.centroid_lat,X.centroid_long))
              
first_chunk = (first_chunk
            >>mutate(lat_long = tuple(zip(first_chunk.latitude,first_chunk.longitude)))
             >>mutate(lat_long_tup =X.lat_long.map(lambda x: (str(x[0]),str(x[1]))))
            >>drop(X.lat_long)
               )
        
first_chunk

,YEAR_BUILT,EMV_BLDG,Shape_Area,ACRES_DEED,OWN_ADD_L1,STREETTYPE,PREFIX_DIR,OWNER_NAME,ZIP4,AG_PRESERV,...,XUSE2_DESC,LANDMARK,OWN_ADD_L3,SCHOOL_DST,TAX_EXEMPT,USE2_DESC,TAX_ADD_L2,latitude,longitude,lat_long_tup
0,0.0,0.0,NaN,30.00,23725 NACRE ST NW,NaN,NaN,JONES TRUSTEE RAYMOND,NaN,N,...,NaN,NaN,"MN, 55330",15,N,NaN,ELK RIVER,45.3977,-93.4622,"(45.39768, -93.46219)"
1,1967.0,138500.0,NaN,8.81,5288 FOREST RIDGE RD,ST,NaN,BRAUSEN RICHARD,NaN,N,...,NaN,NaN,"MN, 56450",15,N,NaN,GARRISON,45.3985,-93.4513,"(45.39852, -93.45134)"
2,2002.0,222400.0,NaN,0.00,23449 VARIOLITE ST NW,ST,NaN,HARVEY JAMES,NaN,N,...,NaN,NaN,"MN, 55330",15,N,NaN,ST FRANCIS,45.3946,-93.4577,"(45.39462, -93.45767)"
3,2001.0,241800.0,NaN,0.00,23309 VARIOLITE ST,ST,NaN,NACHREINER CONNIE,NaN,N,...,NaN,NaN,"MN, 55330",15,N,NaN,ELK RIVER,45.3928,-93.4577,"(45.39283, -93.45771)"
4,1987.0,139100.0,NaN,0.00,23925 GERMANIUM ST NW,ST,NaN,COOPER RONALD,NaN,N,...,NaN,NaN,"MN, 55070",15,N,NaN,ST. FRANCIS,45.4032,-93.407,"(45.40319, -93.40698)"
5,1994.0,157500.0,NaN,0.00,23922 GERMANIUM ST NW,ST,NaN,LUDFORD KELLY L,NaN,N,...,NaN,NaN,"MN, 55070",15,N,NaN,ST. FRANCIS,45.4032,-93.4084,"(45.4032, -93.40839)"
6,2006.0,158100.0,NaN,0.00,5763 244TH CT NW,CT,NaN,HERING CHAD,NaN,N,...,NaN,NaN,"MN, 55070",15,N,NaN,ST. FRANCIS,45.4133,-93.4129,"(45.41326, -93.41293)"
7,2004.0,141200.0,NaN,0.00,5729 244TH CT NW,CT,NaN,SCHWAN JENNIFER,NaN,N,...,NaN,NaN,"MN, 55070",15,N,NaN,SAINT FRANCIS,45.4133,-93.412,"(45.41328, -93.41204)"
8,1996.0,167200.0,NaN,60.78,23946 QUICKSILVER ST NW,ST,NaN,KOGLER JULIE,NaN,N,...,NaN,NaN,"MN, 55070",15,N,NaN,ST FRANCIS,45.4075,-93.4256,"(45.40747, -93.42559)"
9,0.0,0.0,NaN,9.69,23580 NACRE ST NW,NaN,NaN,WATROBA MICHAEL & SANDRA,NaN,N,...,NaN,NaN,"MN, 55330",15,N,NaN,ELK RIVER,45.3964,-93.4527,"(45.39637, -93.45266)"


In [121]:
from project_data_khanal import lat_long_id__lake_dict

(first_chunk >>
         mutate(lake_code = X.lat_long_tup.map(lat_long_id__lake_dict))
         mutate()
 
)

,YEAR_BUILT,EMV_BLDG,Shape_Area,ACRES_DEED,OWN_ADD_L1,STREETTYPE,PREFIX_DIR,OWNER_NAME,ZIP4,AG_PRESERV,...,LANDMARK,OWN_ADD_L3,SCHOOL_DST,TAX_EXEMPT,USE2_DESC,TAX_ADD_L2,latitude,longitude,lat_long_tup,lake_code
0,0.0,0.0,NaN,30.00,23725 NACRE ST NW,NaN,NaN,JONES TRUSTEE RAYMOND,NaN,N,...,NaN,"MN, 55330",15,N,NaN,ELK RIVER,45.3977,-93.4622,"(45.39768, -93.46219)",NaN
1,1967.0,138500.0,NaN,8.81,5288 FOREST RIDGE RD,ST,NaN,BRAUSEN RICHARD,NaN,N,...,NaN,"MN, 56450",15,N,NaN,GARRISON,45.3985,-93.4513,"(45.39852, -93.45134)",NaN
2,2002.0,222400.0,NaN,0.00,23449 VARIOLITE ST NW,ST,NaN,HARVEY JAMES,NaN,N,...,NaN,"MN, 55330",15,N,NaN,ST FRANCIS,45.3946,-93.4577,"(45.39462, -93.45767)",NaN
3,2001.0,241800.0,NaN,0.00,23309 VARIOLITE ST,ST,NaN,NACHREINER CONNIE,NaN,N,...,NaN,"MN, 55330",15,N,NaN,ELK RIVER,45.3928,-93.4577,"(45.39283, -93.45771)",NaN
4,1987.0,139100.0,NaN,0.00,23925 GERMANIUM ST NW,ST,NaN,COOPER RONALD,NaN,N,...,NaN,"MN, 55070",15,N,NaN,ST. FRANCIS,45.4032,-93.407,"(45.40319, -93.40698)",NaN
5,1994.0,157500.0,NaN,0.00,23922 GERMANIUM ST NW,ST,NaN,LUDFORD KELLY L,NaN,N,...,NaN,"MN, 55070",15,N,NaN,ST. FRANCIS,45.4032,-93.4084,"(45.4032, -93.40839)",NaN
6,2006.0,158100.0,NaN,0.00,5763 244TH CT NW,CT,NaN,HERING CHAD,NaN,N,...,NaN,"MN, 55070",15,N,NaN,ST. FRANCIS,45.4133,-93.4129,"(45.41326, -93.41293)",NaN
7,2004.0,141200.0,NaN,0.00,5729 244TH CT NW,CT,NaN,SCHWAN JENNIFER,NaN,N,...,NaN,"MN, 55070",15,N,NaN,SAINT FRANCIS,45.4133,-93.412,"(45.41328, -93.41204)",NaN
8,1996.0,167200.0,NaN,60.78,23946 QUICKSILVER ST NW,ST,NaN,KOGLER JULIE,NaN,N,...,NaN,"MN, 55070",15,N,NaN,ST FRANCIS,45.4075,-93.4256,"(45.40747, -93.42559)",NaN
9,0.0,0.0,NaN,9.69,23580 NACRE ST NW,NaN,NaN,WATROBA MICHAEL & SANDRA,NaN,N,...,NaN,"MN, 55330",15,N,NaN,ELK RIVER,45.3964,-93.4527,"(45.39637, -93.45266)",NaN


In [123]:
first_chunk["lat_long_tup"].iloc[0]

('45.39768', '-93.46219')

In [127]:
lat_long_id__lake_dict

{('45.05462', '-93.09409'): ('62003801-01', 'Vadnais Lake'),
 ('45.1054', '-92.84272'): ('82001501-01', 'Herber Pond'),
 ('44.60604', '-93.56303'): ('70009800-01', 'Pleasant Lake'),
 ('45.07818', '-92.84709'): ('82001900-01', 'South Twin Lake'),
 ('45.22645', '-92.81953'): ('82006800-01', 'Long Lake'),
 ('44.85152', '-93.81634'): ('10008500-01', 'Scheuble Lake'),
 ('44.74112', '-93.26586'): ('19002400-01', 'Wood Lake'),
 ('44.71021', '-93.39857'): ('70002100-01', 'Markley Lake'),
 ('45.1504', '-93.56836'): ('70009800-01', 'Pleasant Lake'),
 ('45.33766', '-93.44717'): ('02013000-01', 'Pickerel Lake'),
 ('44.83769', '-93.58762'): ('10001400-01', 'Hazeltine Lake'),
 ('44.72101', '-93.16325'): ('19045600-01', 'Cobblestone Lake'),
 ('44.71509', '-93.20902'): ('19034800-01', 'Valley Lake'),
 ('44.86285', '-93.89873'): ('10010300-01', 'Berliner Lake'),
 ('44.79129', '-93.60345'): ('10022600-01', "Fireman's Clayhole Lake"),
 ('44.72062', '-93.2452'): ('19044600-01', 'Lac Lavon Lake'),
 ('44.87

In [31]:
first_chunk

,bldg_num,city,county_id,emv_bldg,emv_land,emv_total,homestead,num_units,own_add_l1,own_add_l2,...,tax_capac,tax_exempt,tax_name,total_tax,wshd_dist,year_built,year,zip,centroid_long,centroid_lat
0,NaN,SAINT FRANCIS,3,0.0,132100.0,132100.0,N,0,23725 NACRE ST NW,ELK RIVER,...,1080.0,N,JONES TRUSTEE RAYMOND,1410.0,UPPER RUM RIVER WMO,0.0,2015,55330,-93.46219,45.39768
1,23640.0,SAINT FRANCIS,3,138500.0,57200.0,195700.0,Y,0,5288 FOREST RIDGE RD,GARRISON,...,1807.0,N,BRAUSEN RICHARD,2666.0,UPPER RUM RIVER WMO,1967.0,2015,55330,-93.45134,45.39852
2,23449.0,SAINT FRANCIS,3,222400.0,51400.0,273800.0,Y,0,23449 VARIOLITE ST NW,ST FRANCIS,...,2612.0,N,HARVEY JAMES,3593.0,UPPER RUM RIVER WMO,2002.0,2015,55330,-93.45767,45.39462
3,23309.0,SAINT FRANCIS,3,241800.0,51900.0,293700.0,Y,0,23309 VARIOLITE ST,ELK RIVER,...,2829.0,N,NACHREINER CONNIE,3938.0,UPPER RUM RIVER WMO,2001.0,2015,55330,-93.45771,45.39283
4,23925.0,SAINT FRANCIS,3,139100.0,47200.0,186300.0,Y,0,23925 GERMANIUM ST NW,ST. FRANCIS,...,1658.0,N,COOPER RONALD,2353.0,UPPER RUM RIVER WMO,1987.0,2015,55070,-93.40698,45.40319
5,23922.0,SAINT FRANCIS,3,157500.0,47200.0,204700.0,Y,0,23922 GERMANIUM ST NW,ST. FRANCIS,...,1859.0,N,LUDFORD KELLY L,2603.0,UPPER RUM RIVER WMO,1994.0,2015,55070,-93.40839,45.4032
6,5763.0,SAINT FRANCIS,3,158100.0,47200.0,205300.0,Y,0,5763 244TH CT NW,ST. FRANCIS,...,1865.0,N,HERING CHAD,2594.0,UPPER RUM RIVER WMO,2006.0,2015,55070,-93.41293,45.41326
7,5729.0,SAINT FRANCIS,3,141200.0,47200.0,188400.0,Y,0,5729 244TH CT NW,SAINT FRANCIS,...,1681.0,N,SCHWAN JENNIFER,2347.0,UPPER RUM RIVER WMO,2004.0,2015,55070,-93.41204,45.41328
8,23946.0,SAINT FRANCIS,3,167200.0,219500.0,386700.0,Y,0,23946 QUICKSILVER ST NW,ST FRANCIS,...,3695.0,N,KOGLER JULIE,4977.0,UPPER RUM RIVER WMO,1996.0,2015,55070,-93.42559,45.40747
9,NaN,SAINT FRANCIS,3,0.0,51400.0,51400.0,N,0,23580 NACRE ST NW,ELK RIVER,...,257.0,N,WATROBA MICHAEL & SANDRA,181.0,UPPER RUM RIVER WMO,0.0,2015,55330,-93.45266,45.39637


5. Now convert your expression from the last problem to a function and test that this function works on the first few chunks of each file.

6. We need to make a unique primary key for each row in the combined parcel file.<br>
    a. There is a column that appears to be a unique parcel id.  Double check that this is a true primary key for each individual file. (To do this you need to verify that the number of unique values is the same as the number of rows for each of the parcel files.  **Hint:** For each file, use of the accumulator pattern with two accumualtors (one number and one data frame). <br>
    b. Explain why this column will not work as a primary key if we want to combine all years in one database. <br>
    c. Suppose we make a new column that consist of `str(year) + '-' + PID`.  Explain why this should make a proper primary key for the combined data. <br>

7. Make a function to add the key suggested in the last problem (`str(year) + '-' + PID`) to a given chunk.

#### Note: If you are clever, you can do parts 8 in one double loop, which will save you from having to read the parcel files twice.

8. It is probably worth our time to test that our new key column is truely unique. (If not, we might be wasting out time loading the data into a database, only to have process fail hours in.) Test that the new column works by <br>
    a. Iterating over all the files.<br>
    b. Using an accumulator to count total number of rows across all parcel files. <br>
    c. Using an accumulator to accumulate a set of all unique values of our new key. <br>
    d. Verifying that we have as many total rows as unique keys.
    a. Selecting just this column. <br>
    b. Dumping this column into a temporary database <br>

9. If the last step succeeded, you can proceed to make a master parcel data database.  If not, you will need to figure out another primary key, probably an `id` column similar to the example in the lectures.